In [1]:
#! pip install --upgrade folium
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.clients.fdsn import RoutingClient


In [2]:
minlat = 37
maxlat = 50.0
minlg = -5
maxlg = 15
minmag = 3.5

start = UTCDateTime("2018-09-15") # XT
end = UTCDateTime("2019-12-15") #YP

start = UTCDateTime("2012-06-15") # YP
end = UTCDateTime("2013-09-01") #YP


start = UTCDateTime("2011-11-12") # Pyrope est X7 PE* PF*
end = UTCDateTime("2012-04-25") #Pyrope est
start = UTCDateTime("2018-10-04") # cifalps2 XT
end = UTCDateTime("2019-12-01") #cifalps2
start = UTCDateTime("2012-10-20") # Pyrope west X7, PW*+PY15
end = UTCDateTime("2013-10-01") #Pyrope west
start = UTCDateTime("2013-11-01") # Pyrope center X7, PC*+PY09B
end = UTCDateTime("2014-07-01") #Pyrope center
start = UTCDateTime("2018-01-01")
end = UTCDateTime("2025-12-01")



latcent=(minlat+maxlat)/2
loncent=(minlg+maxlg)/2
print(latcent,loncent)

43.5 5.0


In [ ]:
events_emsc = Client("EMSC").get_events(
    minlatitude=minlat,
    maxlatitude=maxlat,
    minlongitude=minlg,
    maxlongitude=maxlg,
    minmagnitude=minmag,
    starttime=start,
    endtime=end
)
print("\nFound %s event(s) from EMSC Data Center:\n" % len(events_emsc))
print(events_emsc)

In [ ]:
client = RoutingClient("eida-routing")
#client=Client("RESIF") #massif central
inventory=client.get_stations(
    channel='HH?,BH?,SH?',
    minlatitude=minlat,
    maxlatitude=maxlat,
    minlongitude=minlg,
    maxlongitude=maxlg,
    starttime=start,
    endtime=end
)
print(inventory)

In [ ]:
events_stations_combine(inventory,events_emsc)

In [ ]:
def events_stations_combine (inventory, events) :

    import folium
    events_combine=folium.Map(location=[latcent,loncent], zoom_start=5)

    for network in inventory.networks:
        for station in network.stations:
            folium.RegularPolygonMarker(
                location=[station.latitude, station.longitude],
                tooltip=f"{network.code}.{station.code}",
                color="red",
                fill=True,
                number_of_sides=3,
                radius=3,
                rotation=30,
            ).add_to(events_combine)

    for event in events:
        for origin, magnitude in zip(event.origins, event.magnitudes):
                lat, lon, depth, mag, tt = (
                    origin.latitude,
                    origin.longitude,
                    origin.depth,
                    magnitude.mag,
                    origin.time
                )
        
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=200 * 2 ** (mag) / 2 ** 10,
                    color='blue',
                    fill=True,
                    fill_color="#FF8C00",
                    tooltip=f"{mag}.{tt}",
                ).add_to(events_combine)
        
    return events_combine